In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
from scipy import signal
import time
import datetime
import serial
from scipy.fftpack import fft,ifft
from matplotlib.pylab import mpl
import struct
import os
import serial

mpl.rcParams['font.sans-serif'] = ['SimHei']
mpl.rcParams['axes.unicode_minus'] = False

In [3]:
# xxxwrj 无人机惯导
def re_strings(data):
    split_num = 2
    crc_data = data
    lists = [crc_data[i*split_num:i*split_num+split_num].upper() for i in range(len(crc_data)//split_num)]
    return ''.join(lists[::-1])
# print(re_strings('0123456789'))

strings = ''
strings+='90EB'
strings+='1600' # 总字节-5

# strings+='0080' #北斗/GNSS 北斗
# strings+='0040' #北斗/GNSS GNSS
# strings+='0020'     # 混合
strings+='00A0' #北斗/GNSS 组合导航

# strings+=re_strings('8000')     # 指令字-对准
strings+=re_strings('8000')     # 指令字-对准

loc1 = 116.50270
loc2 = 39.73155
loc3 = 20
# loc1 = 0
# loc2 = 0
# loc3 = 0
strings+=re_strings(hex(int(loc1/90*1073741824))[2:].rjust(8,'0'))
strings+=re_strings(hex(int(loc2/90*1073741824))[2:].rjust(8,'0'))
strings+=re_strings(hex(int(loc3/16384*1073741824))[2:].rjust(8,'0'))
for i in range(3):
    strings+='0000'


split_num = 2
crc_data = strings
lists = [crc_data[i*split_num:i*split_num+split_num].upper() for i in range(len(crc_data)//split_num)]
int_lists = [int(num,16) for num in lists]
# print(len(int_lists[2:]))
checks = hex(sum(int_lists[2:]))[2:].rjust(2,'0')[-2:]
lists.append(checks.upper())
counts = []
for i in range(27):
    counts.append(str(i).rjust(2,'0'))
print(' '.join(counts))
print(' '.join(lists))
print('总长度:{}'.format(len(lists)))

00 01 02 03 04 05 06 07 08 09 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26
90 EB 16 00 00 A0 00 80 57 AB D8 52 6F E8 40 1C 00 00 14 00 00 00 00 00 00 00 29
总长度:27


In [10]:
90 EB 16 00 00 A0 00 80 57 AB D8 52 6F E8 40 1C 00 00 14 00 00 00 00 00 00 00 29

# 组合导航 BD+GNSS
90 EB 16 00 00 E0 00 80 57 AB D8 52 6F E8 40 1C 00 00 14 00 00 00 00 00 00 00 69
# 组合导航 切换GNSS
90 EB 16 00 00 60 00 80 57 AB D8 52 6F E8 40 1C 00 00 14 00 00 00 00 00 00 00 E9
# 组合导航 切换BD
90 EB 16 00 00 A0 00 80 57 AB D8 52 6F E8 40 1C 00 00 14 00 00 00 00 00 00 00 29

# 纯惯 切换BD
90 EB 16 00 00 80 00 80 57 AB D8 52 6F E8 40 1C 00 00 14 00 00 00 00 00 00 00 09
# 纯惯 切换GNSS
90 EB 16 00 00 40 00 80 57 AB D8 52 6F E8 40 1C 00 00 14 00 00 00 00 00 00 00 C9


'0x509'

In [14]:
1/3.6/16384*2048

0.034722222222222224